In [1]:
''' 
 Best Practices
1. Preprocessign and cleaning
2. Train, Test , Split
3. BOW, TFIDF, Word2Vec
4. Train ML Algorithms

 
'''



' \n Best Practices\n1. Preprocessign and cleaning\n2. Train, Test , Split\n3. BOW, TFIDF, Word2Vec\n4. Train ML Algorithms\n\n \n'

In [2]:
# Load the data
import pandas as pd
data = pd.read_csv('all_kindle_review.csv')


In [3]:
data.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [4]:
data = data[['reviewText', 'rating']]
data.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [5]:
data.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [6]:
data.shape

(12000, 2)

In [7]:
data['rating'].unique()

array([3, 5, 4, 2, 1], dtype=int64)

In [8]:
data['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

In [9]:
## Preprocessing and cleaning
# Positive review is 1 and negative review is 0
data['rating'] = data['rating'].apply(lambda x:0 if x <3 else 1)

In [10]:
data.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1


In [11]:
data['rating'].value_counts()

rating
1    8000
0    4000
Name: count, dtype: int64

In [12]:
## 1. Lower All the cases

data['reviewText'] = data['reviewText'].str.lower()
data.head()

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [13]:
## 2. i) Removing special characters
import re
data['reviewText'] = data['reviewText'].apply(lambda x: re.sub('[^a-z A-z 0-9]+', ' ', x))



In [14]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tomsh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
## ii) remove the stopwords
data['reviewText'] = data['reviewText'].apply(lambda x: " ".join([y for y in x.split() if y not in stopwords.words('english')]))



In [16]:
## iii) Remove URL
data['reviewText']=data['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))



In [17]:
from bs4 import BeautifulSoup

In [18]:
## iv) Remove html tags
data['reviewText'] = data['reviewText'].apply(lambda x: BeautifulSoup(x,'lxml').get_text())

In [19]:
# v) Remove any Additional spaces

data['reviewText'] = data['reviewText'].apply(lambda x: " ".join(x.split()))


In [20]:
data.head()

,reviewText,rating
0,jace rankin may short nothing mess man hauled ...,1
1,great short read want put read one sitting sex...,1
2,start saying first four books expecting 34 con...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


In [21]:
## Lemmatizer
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()


In [22]:
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [23]:
data['reviewText'] = data['reviewText'].apply(lambda x: lemmatize_words(x))

In [24]:
data.head()

,reviewText,rating
0,jace rankin may short nothing mess man hauled ...,1
1,great short read want put read one sitting sex...,1
2,start saying first four book expecting 34 conc...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


In [25]:
# Train test split
from sklearn.model_selection import train_test_split

X_train,X_test, y_train,y_test = train_test_split(data['reviewText'],data['rating'],test_size=0.2)

In [26]:
from sklearn.feature_extraction.text import CountVectorizer

bow = CountVectorizer()
X_train_bow = bow.fit_transform(X_train).toarray()
X_test_bow = bow.transform(X_test).toarray()


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train).toarray()
X_test_tfidf = tfidf.transform(X_test).toarray()

In [28]:
X_train_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [29]:
from sklearn.naive_bayes import GaussianNB
nb_model_bow = GaussianNB()
nb_model_tfidf = GaussianNB()

In [30]:
nb_model_bow.fit(X_train_bow,y_train)

,priors,None
,var_smoothing,1e-09


In [31]:
nb_model_tfidf.fit(X_train_tfidf, y_train)

,priors,None
,var_smoothing,1e-09


In [32]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

y_pred_bow = nb_model_bow.predict(X_test_bow)

y_pred_tfidf = nb_model_tfidf.predict(X_test_tfidf)

In [33]:
print("Bow  accuracy", accuracy_score(y_test, y_pred_bow))
print("Bow  accuracy", accuracy_score(y_test, y_pred_tfidf))

Bow  accuracy 0.5741666666666667
Bow  accuracy 0.5825


In [34]:
print(confusion_matrix(y_test, y_pred_bow))

[[526 283]
 [739 852]]


In [35]:
print(confusion_matrix(y_test, y_pred_tfidf))

[[515 294]
 [708 883]]


In [36]:
print(classification_report(y_test, y_pred_bow))

              precision    recall  f1-score   support

           0       0.42      0.65      0.51       809
           1       0.75      0.54      0.63      1591

    accuracy                           0.57      2400
   macro avg       0.58      0.59      0.57      2400
weighted avg       0.64      0.57      0.59      2400



In [37]:
print(classification_report(y_test, y_pred_tfidf))

              precision    recall  f1-score   support

           0       0.42      0.64      0.51       809
           1       0.75      0.55      0.64      1591

    accuracy                           0.58      2400
   macro avg       0.59      0.60      0.57      2400
weighted avg       0.64      0.58      0.59      2400



In [ ]:
## Word2Vec

from  gensim.models import Word2Vec


In [53]:
# Prepare the data for Word2Vec
tokenized_review =  [review.split() for review in data['reviewText']]

In [54]:
# Train test split for Word2Vec

X_train_word2vec, X_test_word2vec, y_train_word2vec, y_test_word2vec = train_test_split(tokenized_review, data['rating'], test_size=0.2)



In [55]:
w2v_model = Word2Vec(sentences=X_train_word2vec, vector_size=100, window=5, min_count=1, workers=4)

In [56]:
w2v_model.wv['great']

array([-0.5089185 ,  0.36223334,  0.04618561, -0.538312  ,  0.55317587,
       -0.8429551 ,  0.3797362 ,  0.6601477 , -0.1874679 , -0.4793904 ,
       -0.20392087, -1.0715753 , -0.39922294,  0.517951  ,  0.16355608,
       -0.5697598 ,  0.9894163 , -0.95937634, -0.28075266, -1.6745512 ,
        0.7797058 ,  0.50271434,  1.292914  , -0.85376394,  0.36037296,
        0.9543162 , -0.795919  , -0.18914235, -0.82825315,  0.3985345 ,
        1.3458306 , -0.03810042,  0.374411  , -1.3103244 , -1.1165506 ,
        0.6245681 ,  0.11054633,  0.2062773 , -0.9473178 , -1.094803  ,
        0.6757754 , -0.32905847,  0.20260513, -0.25264287,  0.89965767,
       -0.06734801, -0.42859092, -0.6129954 ,  0.19997458,  0.28923312,
        0.5719338 , -0.6096904 , -0.40534973,  0.10679624, -0.88707185,
       -0.04014296,  0.83632356, -0.0780189 , -1.1684147 , -0.02990636,
        0.13680881,  0.2543805 ,  0.42902404,  0.7284555 , -0.28484964,
        1.1268458 ,  0.5343897 ,  0.12888265, -0.6473685 ,  0.89

In [46]:
w2v_model.wv.similar_by_word('great')

[('wonderful', 0.9327167868614197),
 ('enjoyable', 0.9158170223236084),
 ('entertaining', 0.9105780720710754),
 ('fantastic', 0.9021698832511902),
 ('amazing', 0.8910585045814514),
 ('awesome', 0.8864080905914307),
 ('arc', 0.8802453875541687),
 ('excellent', 0.8751329779624939),
 ('exciting', 0.8730446696281433),
 ('satisfying', 0.8721625804901123)]

In [ ]:
# AverageWord2Vec
from gensim.models import Word2Vec

# Tokenize
tokenized_review_avg2vec = [review.split() for review in data['reviewText']]


In [ ]:
# Train test split(Before Word2Vec)

X_train_avgword2vec, X_test_avgword2vec,y_train_avgword2vec,y_test_avgword2vec = train_test_split(tokenized_review_avg2vec, data['rating'], test_size=0.2)

In [65]:
# Train Word2Vec only on training data

avg_w2vec_model = Word2Vec(sentences=X_train_avgword2vec, vector_size=100, window=5, min_count=1, workers=4)

In [66]:
import numpy as np

In [67]:
# AverageWord2Vec Function
def avg_word_vec(review):
       '''
        Get the average Word2Vec embedding for a list of tokens (words in a review).
        If a word isn't in the vocabulary, it is skipped.
        If no words are found, return a zero vector.
       '''
       vectors =[]
       for word in review:
              if word in avg_w2vec_model.wv:
                     vectors.append(avg_w2vec_model.wv[word])
       
       if vectors:
              return np.mean(vectors, axis=0)
       else:
              return np.zeros(avg_w2vec_model.vector_size)

In [69]:
# Convert all Train and Test reviews to average Word2Vec

X_train_avgw2vec = np.array([ avg_word_vec(review) for review in X_train_avgword2vec])
X_test_avgword2vec = np.array([ avg_word_vec(review) for review in X_test_avgword2vec])

In [70]:
# Train a Classifier(Navie Bayes)
model = GaussianNB()
model.fit(X_test_avgword2vec, y_test_avgword2vec)

,priors,None
,var_smoothing,1e-09


In [71]:
# Predict and Evaluate
y_pred_avgw2vec = model.predict(X_test_avgword2vec)

In [72]:
accuracy_score(y_test_avgword2vec, y_pred_avgw2vec)

0.6495833333333333